In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
Khoang_gtri = {}
DS_Nhan = {}


In [ ]:
class NaiveBayesClassifier:
    def __init__(self):
        self.XS_CoDK_kqua = defaultdict(float)
        self.XS_DacTrung = defaultdict(lambda: defaultdict(float))
        self.Gtri_DacTrung = defaultdict(set)
    def fit(self, data, label_col):
        self.classes = data[label_col].unique()
        TongDT = len(data)
        for cls in self.classes:
            class_data = data[data[label_col] == cls]
            self.XS_CoDK_kqua[cls] = len(class_data) / TongDT
            for col in data.columns:
                if col == label_col:
                    continue
                self.Gtri_DacTrung[col].update(data[col].unique())
                for value in self.Gtri_DacTrung[col]:

                    self.XS_DacTrung[(col, value)][cls] = (len(class_data[class_data[col] == value])/ (len(class_data)))

    def predict(self, conditions):
        max_prob = 0
        best_class = None
        for cls in self.classes:
            XS_SK = self.XS_CoDK_kqua[cls]
            for feature, value in conditions.items():

                if value not in self.Gtri_DacTrung[feature]:
                    likelihood = 1 / (len(self.Gtri_DacTrung[feature]) + len(self.classes))
                else:
                    likelihood = self.XS_DacTrung[(feature, value)].get(cls, 1 / (len(self.classes)))
                XS_SK *= likelihood

            if XS_SK > max_prob:
                max_prob = XS_SK
                best_class = cls

        return best_class

In [ ]:
def Tinh_BMI(data, height_col="Height", weight_col="Weight"):
    if data[height_col].max() > 10:
        data[height_col] = data[height_col] / 100.0
    data["BMI"] = data[weight_col] / (data[height_col] ** 2)
    return data


In [ ]:
def PhanLoai_BMI(data):
    bins = [0, 18.5, 24.9, 29.9, float("inf")]
    labels = ["Underweight", "Normal weight", "Overweight", "Obesity"]
    data["BMI_category"] = pd.cut(data["BMI"], bins=bins, labels=labels)
    return data


In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/Data/ObesityDataSet_raw_and_data_sinthetic_shuffled.csv"
data = pd.read_csv(path)

In [ ]:
data = Tinh_BMI(data)
data = PhanLoai_BMI(data)
data = data.drop(columns=["BMI"])
data = data.drop(columns=["Height", "Weight"])
temp_Original_Data = data.copy(deep=True)
x = {}
numeric_cols = data.select_dtypes(include=["float64", "int64"]).columns


In [ ]:
def chia_khoang_cac_cot_so(data):
    for col in numeric_cols:
        bins = pd.qcut(data[col], q=4, duplicates="drop", retbins=True)[1]
        Khoang_gtri[col] = bins
        labels = [f"bin_{i}" for i in range(1, len(bins))]
        DS_Nhan[col] = labels
        data[col] = pd.cut(data[col], bins=bins, labels=labels, include_lowest=True)
    return data

In [ ]:
dataC = chia_khoang_cac_cot_so(data)
label_column = "NObeyesdad"
X = dataC.drop(columns=[label_column])
y = dataC[label_column]

temp_Original_Data = temp_Original_Data.drop(columns=[label_column])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

train_data = pd.concat([X_train, y_train], axis=1)
nb = NaiveBayesClassifier()
nb.fit(train_data, label_col=label_column)


val_conditions = [dict(row) for _, row in X_val.iterrows()]
val_predictions = [nb.predict(conditions) for conditions in val_conditions]


accuracy = accuracy_score(y_val, val_predictions)
report = classification_report(y_val, val_predictions)

print(f"Độ chính xác trên tập xác thực: {accuracy:.2f}")
print("\nBáo cáo phân loại:")
print(report)


Độ chính xác trên tập xác thực: 0.88

Báo cáo phân loại:
                     precision    recall  f1-score   support

Insufficient_Weight       0.99      0.98      0.98        88
      Normal_Weight       0.95      0.98      0.97        86
     Obesity_Type_I       0.88      0.81      0.84       107
    Obesity_Type_II       0.82      0.92      0.87        95
   Obesity_Type_III       0.98      0.97      0.98       101
 Overweight_Level_I       0.75      0.65      0.70        71
Overweight_Level_II       0.75      0.81      0.78        86

           accuracy                           0.88       634
          macro avg       0.88      0.87      0.87       634
       weighted avg       0.88      0.88      0.88       634



In [ ]:
print("\nHãy nhập dữ liệu cho các đặc trưng sau:")
conditions = {}
for col in temp_Original_Data.columns:
    if col == "BMI_category":

        height = float(input("Nhập chiều cao (m): ").strip())
        weight = float(input("Nhập cân nặng (kg): ").strip())
        bmi = weight/(height**2)
        bins = [0, 18.5, 24.9, 29.9, float("inf")]
        labels = ["Underweight", "Normal weight", "Overweight", "Obesity"]
        bmi_category = pd.cut([bmi], bins=bins, labels=labels)[0]
        conditions[col] = bmi_category
        print(f"Chỉ số BMI: {bmi:.2f}")
    elif temp_Original_Data[col].dtype in ["float64", "int64"]:

        value = input(f"Nhập giá trị cho {col}: ").strip()
        conditions[col] = value
    else:
        print(f"- {col} (các giá trị khả dụng: {list(temp_Original_Data[col].unique())}):")
        value = input(f"Nhập giá trị cho {col}: ").strip()
        conditions[col] = value

dulieu = {}
conditions = pd.DataFrame(conditions, index=[0])

for col in numeric_cols:
    conditions[col] = pd.to_numeric(conditions[col], errors='coerce')

    dulieu[col] = pd.cut(conditions[col], bins=Khoang_gtri[col], labels=DS_Nhan[col], include_lowest=True)

dulieu = pd.DataFrame(dulieu)
result = nb.predict(dulieu.iloc[0].to_dict())
print(f"\nKết quả dự đoán: {result}")


Hãy nhập dữ liệu cho các đặc trưng sau:
- Gender (các giá trị khả dụng: ['Female', 'Male']):
Nhập giá trị cho Gender: Male
Nhập giá trị cho Age: 32
- family_history_with_overweight (các giá trị khả dụng: ['yes', 'no']):
Nhập giá trị cho family_history_with_overweight: no
- FAVC (các giá trị khả dụng: ['yes', 'no']):
Nhập giá trị cho FAVC: no
Nhập giá trị cho FCVC: 2
Nhập giá trị cho NCP: 1
- CAEC (các giá trị khả dụng: ['Sometimes', 'Frequently', 'Always', 'no']):
Nhập giá trị cho CAEC: no
- SMOKE (các giá trị khả dụng: ['no', 'yes']):
Nhập giá trị cho SMOKE: yes
Nhập giá trị cho CH2O: 2
- SCC (các giá trị khả dụng: ['no', 'yes']):
Nhập giá trị cho SCC: 2
Nhập giá trị cho FAF: 1
Nhập giá trị cho TUE: 1
- CALC (các giá trị khả dụng: ['Sometimes', 'no', 'Frequently', 'Always']):
Nhập giá trị cho CALC: no
- MTRANS (các giá trị khả dụng: ['Public_Transportation', 'Automobile', 'Walking', 'Motorbike', 'Bike']):
Nhập giá trị cho MTRANS: Bike
Nhập chiều cao (m): 1.65
Nhập cân nặng (kg): 68
C